<a href="https://colab.research.google.com/github/cedro3/ICON/blob/master/ICON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title **set up (about 20 min)**

# get the code of ICON
!git clone https://github.com/cedro3/ICON.git
%cd /content/ICON

# install package
!bash colab.sh

# download data
import gdown
gdown.download('https://drive.google.com/uc?id=1rX8EmsO8SBJcXtYTNjz2Pyk5H2FjgpNJ', 'data.zip', quiet=False)
! unzip data.zip

# import function
from function import *

In [ ]:
#@title **display sample**
display_pic('/content/ICON/examples')

In [ ]:
#@title **make 3D-model**

# setting
model = 'icon-filter'  # icon-filter / pifu / pamir
image_folder = 'image'
filename = '05.jpg' #@param {type:"string"}
reset_folder(image_folder)
shutil.copy('examples/'+filename, image_folder+'/'+filename)

# modeling
!source activate icon && python -m apps.infer -cfg ./configs/{model}.yaml -loop_smpl 100 -loop_cloth 200 -colab -gpu 0 -export_video -in_dir {image_folder}

# clear log
from IPython.display import clear_output
clear_output()

# change codec
video_path = f"/content/ICON/results/{model}/vid/{filename[:-4]}_cloth.mp4"
video_display_path = f"/content/ICON/results/{model}/vid/{filename[:-4]}_display.mp4"
!ffmpeg -r 30 -i $video_path -vcodec libx264 $video_display_path -y -loglevel quiet

# make variation
import cv2
cap = cv2.VideoCapture(video_display_path)
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)

crop1 = 'crop=512:512:'+str(width-512)+':0'
! ffmpeg -y -i $video_display_path -filter:v $crop1 -async 1 tmp1.mp4 -loglevel quiet
crop2 = 'crop='+str(width-1024)+':512:0:0'
! ffmpeg -y -i $video_display_path -filter:v $crop2 -async 1 tmp2.mp4 -loglevel quiet
video_display_path1 = os.path.dirname(video_display_path)+'/'+os.path.splitext(filename)[0]+'_c.mp4'
! ffmpeg -y -i tmp2.mp4 -i tmp1.mp4 -filter_complex "hstack" $video_display_path1 -loglevel quiet

crop3 = 'crop=512:512:'+str(width-1024)+':0'
! ffmpeg -y -i $video_display_path -filter:v $crop3 -async 1 tmp3.mp4 -loglevel quiet
video_display_path2 = os.path.dirname(video_display_path)+'/'+os.path.splitext(filename)[0]+'_b.mp4'
! ffmpeg -y -i tmp2.mp4 -i tmp3.mp4 -filter_complex "hstack" $video_display_path2 -loglevel quiet

# display mp4
display_mp4(video_display_path1)

In [ ]:
#@title **download mp4（chrome）**
from google.colab import files
style = 'cloth' #@param ['full', 'cloth', 'body']
if style == 'full':
  files.download(video_display_path)
if style == 'cloth':
  files.download(video_display_path1)
if style == 'body':
  files.download(video_display_path2)